In [42]:
import numpy as np
import tensorflow
from keras.initializers import TruncatedNormal
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import h5py
from keras.callbacks import ModelCheckpoint
import math
import torch
from torch import nn
import torch.nn.functional as F
import brevitas.nn as qnn
from brevitas.quant import Int8WeightPerTensorFixedPoint as WeightQuant
from brevitas.quant import Int8ActPerTensorFixedPoint as ActQuant
from brevitas.quant import Int8BiasPerTensorFixedPointInternalScaling as BiasQuant
import brevitas.nn as qnn
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import layers

s = np.arange(1, 8)
for snr in s:
    # Load Matlab DataSets
    mat = h5py.File('C:\\Users\\yasht\\Desktop\\SAUH Code\\dataset\\Dataset_{}.mat'.format(snr), 'r')
    X = np.array(mat['Preamble_Error_Correction_Dataset']['X'])
    Y = np.array(mat['Preamble_Error_Correction_Dataset']['Y'])
    print('Loaded Dataset Inputs: ', X.shape)
    print('Loaded Dataset Outputs: ', Y.shape)



Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)


In [22]:
qnn.QuantConv2d

brevitas.nn.quant_conv.QuantConv2d

In [23]:
    # Normalizing Datasets
    scalerx = StandardScaler()
    scalerx.fit(X)
    scalery = StandardScaler()
    scalery.fit(Y)
    XS = scalerx.transform(X)
    YS = scalery.transform(Y)
    # save the mean and standard deviation values to file
    a = np.zeros(104)
    b = np.zeros(104)
    for i in range(scalerx.mean_.shape[0]):
        a[i] = scalerx.mean_[i]
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\mean.dat'.format(snr), a, fmt='%f', newline=',')
    for i in range(scalerx.mean_.shape[0]):
        b[i] = math.sqrt(scalerx.var_[i])
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\std.dat'.format(snr), b, fmt='%f', newline=',')
    for i in range(scalery.mean_.shape[0]):
        a[i] = scalery.mean_[i]
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\mean_o.dat'.format(snr), a, fmt='%f', newline=',')
    for i in range(scalery.mean_.shape[0]):
        b[i] = math.sqrt(scalery.var_[i])
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\std_o.dat'.format(snr), b, fmt='%f', newline=',')

   

In [24]:
# Split Data into train and test sets
seed = 7
train_X, test_X, train_Y, test_Y = train_test_split(XS, YS, test_size=0.2, random_state=seed)
print('Training samples: ', train_X.shape[0])
print('Testing samples: ', test_X.shape[0])



Training samples:  1600
Testing samples:  400


In [60]:
# Build the model.
init = TruncatedNormal(mean=0.0, stddev=0.05, seed=None)

class Quantization(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class DPULeNet(Quantization):
    def __init__(self):
        super(DPULeNet, self).__init__()
        self.conv1 = qnn.QuantConv2d(
            3, 6, 5, input_quant=ActQuant, weight_quant=WeightQuant,
            output_quant=ActQuant, bias_quant=BiasQuant, return_quant_tensor=True)
        self.relu1 = nn.ReLU()
        self.conv2 = qnn.QuantConv2d(
            6, 16, 5, weight_quant=WeightQuant, output_quant=ActQuant,
            bias_quant=BiasQuant, return_quant_tensor=True)
        self.relu2 = nn.ReLU()
        self.fc1 = qnn.QuantLinear(
            16*5*5, 120, bias=True, weight_quant=WeightQuant,
            bias_quant=BiasQuant, output_quant=ActQuant, return_quant_tensor=True)
        self.relu3 = nn.ReLU()
        self.fc2 = qnn.QuantLinear(
            120, 84, bias=True, weight_quant=WeightQuant,
            bias_quant=BiasQuant, output_quant=ActQuant, return_quant_tensor=True)
        self.relu4 = nn.ReLU()
        self.fc3 = qnn.QuantLinear(
            84, 10, bias=False, weight_quant=WeightQuant, output_quant=ActQuant)

    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.reshape(out.shape[0], -1)
        out = self.relu3(self.fc1(out))
        out = self.relu4(self.fc2(out))
        out = self.fc3(out)
        return out

    
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = torch.optim.Adam(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history




model_path = 'C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\LS_DNN_{}.h5'.format(snr)



In [61]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.001

In [62]:
    # This check point saves the best DNN model with highest validation accuracy
    checkpoint = ModelCheckpoint(model_path, monitor='val_acc',
                                 verbose=1, save_best_only=True,
                                 mode='max')
    callbacks_list = [checkpoint]
    # Train the model.
    epoch = 500
    batch_size = 32

    model.fit(train_X, train_Y, epochs=epoch, batch_size=batch_size, verbose=2, validation_split=0.25,  callbacks=callbacks_list)


Epoch 1/500

Epoch 00001: val_acc improved from -inf to 0.74250, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 163ms/epoch - 4ms/step
Epoch 2/500

Epoch 00002: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 87ms/epoch - 2ms/step
Epoch 3/500

Epoch 00003: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 87ms/epoch - 2ms/step
Epoch 4/500

Epoch 00004: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 104ms/epoch - 3ms/step
Epoch 5/500

Epoch 00005: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 85ms/epoch - 2ms/step
Epoch 6/500

Epoch 00006: val_acc did not improve from 0.74250
38/38 

Epoch 48/500

Epoch 00048: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 106ms/epoch - 3ms/step
Epoch 49/500

Epoch 00049: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 78ms/epoch - 2ms/step
Epoch 50/500

Epoch 00050: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 69ms/epoch - 2ms/step
Epoch 51/500

Epoch 00051: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 69ms/epoch - 2ms/step
Epoch 52/500

Epoch 00052: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 76ms/epoch - 2ms/step
Epoch 53/500

Epoch 00053: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - v

Epoch 96/500

Epoch 00096: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 96ms/epoch - 3ms/step
Epoch 97/500

Epoch 00097: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 80ms/epoch - 2ms/step
Epoch 98/500

Epoch 00098: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 71ms/epoch - 2ms/step
Epoch 99/500

Epoch 00099: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 97ms/epoch - 3ms/step
Epoch 100/500

Epoch 00100: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 100ms/epoch - 3ms/step
Epoch 101/500

Epoch 00101: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 -

Epoch 144/500

Epoch 00144: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 73ms/epoch - 2ms/step
Epoch 145/500

Epoch 00145: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 69ms/epoch - 2ms/step
Epoch 146/500

Epoch 00146: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 79ms/epoch - 2ms/step
Epoch 147/500

Epoch 00147: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 79ms/epoch - 2ms/step
Epoch 148/500

Epoch 00148: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 70ms/epoch - 2ms/step
Epoch 149/500

Epoch 00149: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-0


Epoch 00191: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 96ms/epoch - 3ms/step
Epoch 192/500

Epoch 00192: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 88ms/epoch - 2ms/step
Epoch 193/500

Epoch 00193: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 90ms/epoch - 2ms/step
Epoch 194/500

Epoch 00194: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 102ms/epoch - 3ms/step
Epoch 195/500

Epoch 00195: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 65ms/epoch - 2ms/step
Epoch 196/500

Epoch 00196: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 

...


Epoch 00285: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 86ms/epoch - 2ms/step
Epoch 286/500

Epoch 00286: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 113ms/epoch - 3ms/step
Epoch 287/500

Epoch 00287: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 114ms/epoch - 3ms/step
Epoch 288/500

Epoch 00288: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 109ms/epoch - 3ms/step
Epoch 289/500

Epoch 00289: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 116ms/epoch - 3ms/step
Epoch 290/500

Epoch 00290: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_ac

Epoch 333/500

Epoch 00333: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 94ms/epoch - 2ms/step
Epoch 334/500

Epoch 00334: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 115ms/epoch - 3ms/step
Epoch 335/500

Epoch 00335: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 95ms/epoch - 3ms/step
Epoch 336/500

Epoch 00336: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 62ms/epoch - 2ms/step
Epoch 337/500

Epoch 00337: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 56ms/epoch - 1ms/step
Epoch 338/500

Epoch 00338: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-

Epoch 380/500

Epoch 00380: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 113ms/epoch - 3ms/step
Epoch 381/500

Epoch 00381: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 113ms/epoch - 3ms/step
Epoch 382/500

Epoch 00382: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 82ms/epoch - 2ms/step
Epoch 383/500

Epoch 00383: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 95ms/epoch - 3ms/step
Epoch 384/500

Epoch 00384: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 80ms/epoch - 2ms/step
Epoch 385/500

Epoch 00385: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e

Epoch 427/500

Epoch 00427: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 91ms/epoch - 2ms/step
Epoch 428/500

Epoch 00428: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 104ms/epoch - 3ms/step
Epoch 429/500

Epoch 00429: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 105ms/epoch - 3ms/step
Epoch 430/500

Epoch 00430: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 102ms/epoch - 3ms/step
Epoch 431/500

Epoch 00431: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 78ms/epoch - 2ms/step
Epoch 432/500

Epoch 00432: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554

Epoch 474/500

Epoch 00474: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 51ms/epoch - 1ms/step
Epoch 475/500

Epoch 00475: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 51ms/epoch - 1ms/step
Epoch 476/500

Epoch 00476: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 57ms/epoch - 1ms/step
Epoch 477/500

Epoch 00477: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 52ms/epoch - 1ms/step
Epoch 478/500

Epoch 00478: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-04 - val_acc: 0.7425 - 58ms/epoch - 2ms/step
Epoch 479/500

Epoch 00479: val_acc did not improve from 0.74250
38/38 - 0s - loss: 9.7863e-04 - acc: 0.7325 - val_loss: 9.9554e-0